# Final Project Results and Metrics

In [1]:
import random
import os
import matplotlib
import pandas as pd
#import the models
import baseline
import nn_with_rules
import base_nn

random.seed(0) # set a seed for reproducibility

c:\Users\mcg19\miniconda3\envs\UMR\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Run Models or Load in from output


In [2]:
baseline_output = "output/baseline_test.csv"
base_nn_output = "output/base_nn_test.csv"
nn_with_rules_output = "output/nn_with_rules_test.csv"

num_iters = 20 # number of times to run the baseline model if not loading it in

#get baseline (just rules based results)
if os.path.exists(baseline_output):
    baseline_df = pd.read_csv(baseline_output)
else:
    baseline_df= baseline.run_baseline(num_iters)
#get base_nn results
if os.path.exists(base_nn_output):
    base_nn_df = pd.read_csv(base_nn_output)
else:
    base_nn_df = base_nn.run_base_nn()
#get nn_with_rules results
if os.path.exists(nn_with_rules_output):
    nn_with_rules_df = pd.read_csv(nn_with_rules_output)
else:
    nn_with_rules_df = nn_with_rules.run_nn_with_rules()
 

### Make sure that the results dfs are comparable

In [3]:
print(baseline_df.shape) # TODO: figure out why they aren't the same size, doing a different preprocessing approach for this
print(base_nn_df.shape)
print(nn_with_rules_df.shape)

(65, 21)
(65, 21)
(65, 21)
